In [29]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from stempel import StempelStemmer

In [30]:
dataset = pd.read_csv('criminal.csv')

In [31]:
def transform_text(text):
    text = text.lower()
    text = text.split()
    ss = StempelStemmer.default()
    text = [ss.stem(word) for word in text]
    if None in text:
        text.remove(None)
    text = ' '.join(text)
    return text

In [32]:
corpus = []
for i in range(dataset.shape[0]):
    art = transform_text(dataset['crime'][i])
    corpus.append(art)

In [34]:
transformed_criminal = pd.DataFrame(corpus, columns=['crime'])
transformed_criminal

,crime
0,§ 1. kto wszczyna lub prowadzić wojnę napastni...
1,§ 3. kto publiczny nawoływać do wszcząć wojny ...
2,"§ 1. kto, w ć wyniszczeć w całość alć w część ..."
3,"§ 2. kto, w ć wyniszczeć w całość alć w część ..."
4,"§ 1. kto, brać udzieć w masowy zamach lub choć..."
...,...
69,kto nieumyślny powodować śmierć człowieić
70,§ 1. kto powodować ciężki uszczerbka na zdrowi...
71,§ 1. kto naywać inny osoba do spowodować u on ...
72,§ 2. kto przemoca lub groźba bezprawny zmuszać...


In [35]:
transformed_criminal.to_csv('transformed.csv', sep=',')

In [36]:
def fit_crime(my_crime, data):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)
    crime_vector = tfidf_vectorizer.transform([my_crime])
    similarity = cosine_similarity(crime_vector, tfidf_matrix)
    print(similarity)
    best_match_index = similarity.argsort()[0][-1]
    return dataset.iloc[best_match_index]['article_number'], dataset.iloc[best_match_index]['crime'], dataset.iloc[best_match_index]['penalty']

In [39]:
wykroczenie_input = "zabiłem człowieka"
wykroczenie_input = transform_text(wykroczenie_input)
print(wykroczenie_input)
numer, opis, kara = fit_crime(wykroczenie_input, corpus)

print(f"Dopasowane przestępstwo: {opis}")
print(f"Grożąca kara: {kara}")

zabić człowieić
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.16098347 0.
  0.         0.         0.         0.         0.         0.
  0.66333101 0.         0.         0.25909804 0.         0.21582128
  0.         0.27259603 0.25074646 0.         0.         0.
  0.         0.         0.         0.39760501 0.09376064 0.
  0.         0.08833605]]
Dopasowane przestępstwo: § 1. Kto zabija człowieka
Grożąca kara: od 10 lat do dożywotniego pozbawienia wolności
